# Visualización cartográfica (Cartographic Visualization)

_“La elaboración de mapas es uno de los esfuerzos intelectuales más antiguos de la humanidad y también uno de los más complejos, con la teoría científica, la representación gráfica, los conocimientos geográficos y las consideraciones prácticas combinadas en una variedad interminable de formas.”_ &mdash; [H. J. Steward](https://books.google.com/books?id=cVy1Ms43fFYC)
 
Cartografía &ndash; el estudio y la práctica de elaborar mapas &ndash; tiene una rica historia que abarca siglos de descubrimiento y diseño. La visualización cartográfica aprovecha las técnicas de la cartografía para transmitir datos que contienen información espacial, como lugares, rutas o trayectorias en la superficie de la Tierra.

<div style="float: right; margin-left: 1em; margin-top: 1em;"><img width="300px" src="https://gist.githubusercontent.com/jheer/c90d582ef5322582cf4960ec7689f6f6/raw/8dc92382a837ccc34c076f4ce7dd864e7893324a/latlon.png" /></div>

Al aproximar la Tierra como una esfera, podemos denotar posiciones utilizando un sistema de coordenadas esféricas de _latitud_ (ángulo en grados al norte o al sur del _Ecuador_) y _longitud_ (ángulo en grados que especifica la posición este-oeste). En este sistema, un _paralelo_ es un círculo de latitud constante y un _meridiano_ es un círculo de longitud constante. El [_Meridiano cero_](https://es.wikipedia.org/wiki/Meridiano_cero) se encuentra a 0° de longitud y por convención se define su paso por el "Royal Observatory" en Greenwich, Inglaterra.

Para "aplanar" una esfera tridimensional en un plano bidimimensional, debemos aplicar una [proyección](https://es.wikipedia.org/wiki/Proyecci%C3%B3n_cartogr%C3%A1fica) que mapea el par (`longitude`, `latitude`)/(`longitud`, `latitud`) a las coordenadas (`x`, `y`). Similar a [las escalas](https://github.com/SimplificandoDatos/Altair/blob/master/04_escalas_ejes_leyendas_altair.ipynb), las proyecciones mapean desde un dominio de datos (posición espacial) hacia un rango visual (posición de píxeles). Sin embargo, las escalas que hemos visto hasta ahora aceptan un dominio unidimensional, mientras que las proyecciones del mapa son inherentemente bidimensionales.

En este *notebook*, introduciremos lo básico de la creación de mapas y la visualización de datos espaciales con Altair, incluyendo:

- Formatos de datos para representar características geográficas,
- Técnicas de geovisualización como mapas con puntos, símbolos y [coropléticos](https://es.wikipedia.org/wiki/Mapa_coropl%C3%A9tico)
- Una revisión de las proyecciones cartográficas más comunes.

_Este notebook es una traduccion y modificación del notebook **"Introduction to Vega-Lite / Altair"** , el cual es parte de [data visualization curriculum](https://github.com/uwdata/visualization-curriculum)._

_Todos los notebooks en español se encuentran en: [Altair en Simplificando Datos](https://github.com/SimplificandoDatos/Altair)_

In [1]:
import pandas as pd
import altair as alt
from vega_datasets import data

## Datos geográficos: GeoJSON y TopoJSON

A este punto, hemos trabajado con conjuntos de datos que se encontraban en archivos JSON y CSV que corresponden a datos tabulares hechos de filas (registros) y columnas. Para representar regiones geográficas (países, estados,  _etc._) y trayectorias ( rutas de vuelos, líneas de subterráneos, _etc._), necesitamos expandir nuestro repertorio con formatos adicionales, que están diseñados para soportar geometrías ricas.

[GeoJSON](https://es.wikipedia.org/wiki/GeoJSON)  modela características geográficas dentro de un formato especializado JSON. Una  `feature` (característica) en GeoJSON puede incluir datos geométricos &ndash; como coordenas en `longitude` (longitud), `latitude` (latitud) para una frontera de un país &ndash; como támbién datos de atributos adcionales.

A continuación se muestra objeto `feature` de GeoJSON para la frontera del estado de Colorado en E.E.U.U:

~~~ json
{
  "type": "Feature",
  "id": 8,
  "properties": {"name": "Colorado"},
  "geometry": {
    "type": "Polygon",
    "coordinates": [
      [[-106.32056285448942,40.998675790862656],[-106.19134826714341,40.99813863734313],[-105.27607827344248,40.99813863734313],[-104.9422739227986,40.99813863734313],[-104.05212898774828,41.00136155846029],[-103.57475287338661,41.00189871197981],[-103.38093099236758,41.00189871197981],[-102.65589358559272,41.00189871197981],[-102.62000064466328,41.00189871197981],[-102.052892177978,41.00189871197981],[-102.052892177978,40.74889940428302],[-102.052892177978,40.69733266640851],[-102.052892177978,40.44003613055551],[-102.052892177978,40.3492571857556],[-102.052892177978,40.00333031918079],[-102.04930288388505,39.57414465707943],[-102.04930288388505,39.56823596836465],[-102.0457135897921,39.1331416175485],[-102.0457135897921,39.0466599009048],[-102.0457135897921,38.69751011321283],[-102.0457135897921,38.61478847120581],[-102.0457135897921,38.268861604631],[-102.0457135897921,38.262415762396685],[-102.04212429569915,37.738153927339205],[-102.04212429569915,37.64415206142214],[-102.04212429569915,37.38900413964724],[-102.04212429569915,36.99365914927603],[-103.00046581851544,37.00010499151034],[-103.08660887674611,37.00010499151034],[-104.00905745863294,36.99580776335414],[-105.15404227428235,36.995270609834606],[-105.2222388620483,36.995270609834606],[-105.7175614468747,36.99580776335414],[-106.00829426840322,36.995270609834606],[-106.47490250048605,36.99365914927603],[-107.4224761410235,37.00010499151034],[-107.48349414060355,37.00010499151034],[-108.38081766383978,36.99903068447129],[-109.04483707103458,36.99903068447129],[-109.04483707103458,37.484617466122884],[-109.04124777694163,37.88049961001363],[-109.04124777694163,38.15283644441336],[-109.05919424740635,38.49983761802722],[-109.05201565922046,39.36680339854235],[-109.05201565922046,39.49786885730673],[-109.05201565922046,39.66062637372313],[-109.05201565922046,40.22248895514744],[-109.05201565922046,40.653823231326896],[-109.05201565922046,41.000287251421234],[-107.91779872584989,41.00189871197981],[-107.3183866123281,41.00297301901887],[-106.85895696843116,41.00189871197981],[-106.32056285448942,40.998675790862656]]
    ]
  }
}
~~~

La `feature` incluye un objeto `properties`, el cual puede incluir cualquier cantidad de campos de datos, más un objeto `geometry`, el cual en este caso incluye un único polígono que consiste en coordenadas de `[longitude, latitude]` de la frontera del estado.

Para saber más acerca de los detalles esenciales de GeoJSON, mira la [especificación oficial de GeoJSON](http://geojson.org/) o lee [el útil manual de Tom MacWright](https://macwright.org/2015/03/23/geojson-second-bite).

Una desventaja de GeoJSON como formato de almacenamiento es que puede ser redundante, lo que resulta en tamaños de archivo más grandes. Consideremos: Colorado comparte fronteras con otros seis estados (siete si incluyes la esquina que toca a Arizona). En lugar de usar listas de coordenadas separadas y superpuestas para cada uno de esos estados, un enfoque más compacto es codificar las fronteras compartidas una sola vez, representando la _topology (topología)_ de las regiones geográficas. ¡Afortunadamente, esto es precisamente lo que hace el formato [TopoJSON](https://github.com/topojson/topojson/blob/master/README.md)!


Carguemos un archivo TopoJSON de los países del  mundo (en una resolución de 110 metros):

In [2]:
world = data.world_110m.url
world

'https://vega.github.io/vega-datasets/data/world-110m.json'

In [3]:
world_topo = data.world_110m()

In [4]:
world_topo.keys()

dict_keys(['type', 'transform', 'objects', 'arcs'])

In [5]:
world_topo['type']

'Topology'

In [6]:
world_topo['objects'].keys()

dict_keys(['land', 'countries'])

_Inspecciona el objeto diccionario de TopoJSON `world_topo` de arriba para ver su contenido._

En los datos anteriores, la propiedad `objects` indica los elementos nombrados que podemos extraer de los datos: geometrías para todos los `countries` (paises), o un único polígono que representa todas las `land` (tierras) de la Tierra. Cualquiera de estos puede ser desempaquetado en datos GeoJSON que podemos visualizar.

Como TopoJSON es un formato especializado, necesitamos instruir a Altair para que analice el formato TopoJSON, indicando el nombre de la `feature` del objeto  que deseamos extraer de la topología. El siguiente código indica que queremos extraer la `feature` de GeoJSON del conjunto de datos `world` para el objeto `countries`:


~~~ js
alt.topo_feature(world, 'countries')
~~~

Este llamado al método `alt.topo_feature` corresponde al siguiente Vega-Lite JSON:

~~~ json
{
  "values": world,
  "format": {"type": "topojson", "feature": "countries"}
}
~~~

¡Ahora que podemos cargar datos geográficos, estamos listos para iniciar con la elaboración de mapas!

## Marcadores Geoshape (Geoshape Marks)

Para visualizar datos geográficos, Altair provee el tipo de marcador `geoshape`. Para crear un mapa básico, podemos crear un marcador `geoshape` y pasarle nuestros datos de TopoJSON, que luego se desempaquetan en *features (características)* de GeoJSON, una para cada país del mundo:

In [7]:
alt.Chart(alt.topo_feature(world, 'countries')).mark_geoshape()

alt.Chart(...)

En el ejemplo anterior, Altair aplica un color azul por defecto y utiliza una proyección de mapa por defecto (`mercator`). Podemos personalizar los colores y los anchos de trazo de los límites usando las propiedades de los marcadores estándar. Usando el método `project` también podemos añadir nuestra propia proyección de mapa:

In [8]:
alt.Chart(alt.topo_feature(world, 'countries')).mark_geoshape(
    fill='#2a1d0c', stroke='#706545', strokeWidth=0.5
).project(
    type='mercator'
)

alt.Chart(...)

Por defecto, Altair ajusta automáticamente la proyección para que todos los datos encajen dentro del ancho y alto de la gráfica. También podemos especificar los parámetros de proyección, como `scale` (nivel de zoom) y `translate` (paneo), para personalizar los ajustes de la proyección. Aquí ajustamos los parámetros de "escala" y "traducir" para enfocarnos en Latinoamérica:

***Nota de Simplificando Datos***: En el notebook orignal se enfocan en Europa, pero lo cambie a Latinoamérica ;)

In [9]:
alt.Chart(alt.topo_feature(world, 'countries')).mark_geoshape(
    fill='#2a1d0c', stroke='#706545', strokeWidth=0.5
).project(
    type='mercator', scale=165, translate=[455, 100]
)

alt.Chart(...)

_Nota como una resolución de 110 m de los datos se hace evidente a esta escala. Para ver líneas y límites costeros más detallados, necesitamos un archivo de entrada con geometrías más finas. ¡Ajusta los parámetros `scale` y `translate` para enfocar el mapa en otras regiones!_

Hasta ahora nuestro mapa sólo muestra países. Usando el operador `layer`, podemos combinar múltiples elementos del mapa. Altair incluye _data generators (generadores de datos)_ que podemos usar para crear datos para capas adicionales del mapa:

- El generador de esferas (`{'sphere': True}`) proporciona una representación en GeoJSON de la esfera completa de la Tierra. Podemos crear un marcador adicional `geoshape` que llena la forma de la Tierra como una capa de fondo.
- El generador de gratículas (`{'graticule': ...}`) crea un *feature* de GeoJSON que representa una _graticule (gratícula)_: una cuadrícula formada por líneas de latitud y longitud. La gratícula por defecto tiene meridianos y paralelos cada 10° entre ±80° de latitud. Para las regiones polares, hay meridianos cada 90°. Estos ajustes pueden ser personalizados usando las propiedades `stepMinor` y `stepMajor`.

Vamos a poner en capas la esfera, la gratícula y los marcadores de país en una especificación de mapa reutilizable:

In [10]:
map = alt.layer(
    # usar la esfera de la Tierra como la capa base
    alt.Chart({'sphere': True}).mark_geoshape(
        fill='#e6f3ff'
    ),
    # añadir una cuadrícula para las líneas de referencia geográfica
    alt.Chart({'graticule': True}).mark_geoshape(
        stroke='#ffffff', strokeWidth=1
    ),
    # y luego los países del mundo
    alt.Chart(alt.topo_feature(world, 'countries')).mark_geoshape(
        fill='#2a1d0c', stroke='#706545', strokeWidth=0.5
    )
).properties(
    width=600,
    height=400
)

Podemos extender el mapa con una proyección deseada y dibujar el resultado. Aquí aplicamos una [proyección natural de la Tierra](https://es.wikipedia.org/wiki/Proyecci%C3%B3n_Natural_Earth). La capa _sphere_ proporciona el fondo azul claro; la capa _graticule_ proporciona las líneas blancas de referencia geográfica.

In [11]:
map.project(
    type='naturalEarth1', scale=110, translate=[300, 200]
).configure_view(stroke=None)

alt.LayerChart(...)

## Point Maps (Mapas con puntos)

Además de los datos _geométricos_ provistos por los archivos GeoJSON o TopoJSON, muchos datos tabulares incluyen información geográfica en la forma de campos coordenadas de `longitude` y `latitude`, o referencia a regiones geográficas como nombres de países, nombres de estados, códigos postales, _etc._, los cuales pueden ser mapeados a coordenadas usando un [servicio de geocodificación](https://es.wikipedia.org/wiki/Geocodificaci%C3%B3n). En algunos casos, los datos de localización son lo suficientemente ricos como para que podamos ver patrones significativos proyectando sólo los puntos de datos &mdash; ¡no se requiere un mapa base!

Veamos un conjunto de datos de códigos postales de 5 dígitos en los Estados Unidos, incluyendo coordinadas de `longitude`, `latitude` para cada oficina de correos, además de un campo `zip_code`.

In [12]:
zipcodes = data.zipcodes.url
zipcodes

'https://vega.github.io/vega-datasets/data/zipcodes.csv'

Podemos visualizar la localización de cada oficina de correo usando un pequeño (1-pixel) marcador `square`. Sin embargo, para ajustar la posición _no_ usamos los canales `x` y `y`. _¿Por qué?_

Mientras que las proyecciones cartográficas trazan las coordenadas (`longitude`, `latitude`) hacia las coordenadas (`x`, `y`), pueden hacerlo de manera arbitraria. ¡No hay garantía, por ejemplo, de que la `longitude` → `x` y la `latitude` → `y`! En su lugar, Altair incluye canales especiales de codificación de `longitude` y `latitude` para manejar las coordenadas geográficas. Estos canales indican qué campos de datos deben ser mapeados a las coordenadas de `longitude` y `latitude`, y luego aplica una proyección para mapear esas coordenadas a las posiciones `x` y `y`.

In [13]:
alt.Chart(zipcodes).mark_square(
    size=1, opacity=1
).encode(
    longitude='longitude:Q', # aplica el campo llamado 'longitude' al canal longitude
    latitude='latitude:Q'    # aplica el campo llamado'latitude' al canal latitude
).project(
    type='albersUsa'
).properties(
    width=900,
    height=500
).configure_view(
    stroke=None
)

alt.Chart(...)

_¡Sólo trazando los códigos postales, podemos ver el contorno de los Estados Unidos y discernir patrones significativos con la densidad de las oficinas de correos, sin un mapa base o elementos de referencia adicionales!_

Usamos la proyección `albersUsa`, que se toma algunas libertades con la geometría real de la Tierra, con versiones a escala de Alaska y Hawai en la esquina inferior izquierda. Como no especificamos los parámetros `scale` o `translate` de la proyección, Altair los ajusta automáticamente para que se ajusten a los datos visualizados. 

Ahora podemos seguir haciendo más preguntas sobre nuestro conjunto de datos. Por ejemplo, ¿hay alguna explicación para la asignación de los códigos postales? Para evaluar esta pregunta podemos añadir una codificación de color basada en el primer dígito del código postal. Primero agregamos una transformación `calculate` para extraer el primer dígito, y codificar el resultado usando el canal de color:

In [14]:
alt.Chart(zipcodes).transform_calculate(
    digit='datum.zip_code[0]'
).mark_square(
    size=2, opacity=1
).encode(
    longitude='longitude:Q',
    latitude='latitude:Q',
    color='digit:N'
).project(
    type='albersUsa'
).properties(
    width=900,
    height=500
).configure_view(
    stroke=None
)

alt.Chart(...)

_Para hacer *zoom* en un dígito específico, agrega una transformación de filtro para limitar los datos que se muestran. Intenta añadir una [selección interactiva](https://github.com/SimplificandoDatos/Altair/blob/master/06_interaccion_altair.ipynb) para filtrar a un solo dígito y actualizar dinámicamente el mapa. Y asegúrate de usar caracteres (\`'1'\`) en lugar de números (\`1\`) cuando estés filtrando los valores de los dígitos._

(¡Este ejemplo está insparado en la visualización clásica [zipdecode](https://benfry.com/zipdecode/) de Ben Fry!)

Podríamos preguntarnos qué podría indicar la _secuencia_ de los códigos postales. Una forma de explorar esta pregunta es conectar cada código postal consecutivo usando un marcador `line`, como lo hace Robert Kosara en la visaulización [ZipScribble](https://eagereyes.org/zipscribble-maps/united-states):

In [15]:
alt.Chart(zipcodes).transform_filter(
    '-150 < datum.longitude && 22 < datum.latitude && datum.latitude < 55'
).transform_calculate(
    digit='datum.zip_code[0]'
).mark_line(
    strokeWidth=0.5
).encode(
    longitude='longitude:Q',
    latitude='latitude:Q',
    color='digit:N',
    order='zip_code:O'
).project(
    type='albersUsa'
).properties(
    width=900,
    height=500
).configure_view(
    stroke=None
)

alt.Chart(...)

_Ahora podemos ver cómo los códigos postales se agrupan más en áreas más pequeñas, indicando una asignación jerárquica de los códigos por ubicación, pero con una notable variabilidad dentro de los grupos locales._

Si prestaste atención a nuestros mapas anteriores, habrás notado que hay códigos postales trazados en la esquina superior izquierda. Estos corresponden a lugares como Puerto Rico o Samoa Americana, que contienen códigos postales de EE.UU. pero están mapeados en coordenadas `null` (`0`, `0`) por la proyección `albersUsa`. Además, Alaska y Hawai pueden complicar nuestra visión de los segmentos de línea de conexión. En respuesta, el código de arriba incluye un filtro adicional que elimina los puntos fuera de nuestra `longitude` y `latitude` elegidas.

_¡Quita el filtro de arriba para ver qué pasa!_

## Symbol Maps (Mapas con símbolos)

Ahora combinemos un mapa base y datos graficados como capas separadas. Examinaremos la red de vuelos comerciales de EE.UU., considerando tanto los aeropuertos como las rutas de vuelo. Para ello, necesitaremos tres conjuntos de datos.
Para nuestro mapa base, usaremos un archivo TopoJSON para los Estados Unidos con una resolución de 10m, que contiene características (*features*) para `states` o `counties`:

In [16]:
usa = data.us_10m.url
usa

'https://vega.github.io/vega-datasets/data/us-10m.json'

Para los aeropuertos, usaremos un conjunto de datos con campos para las coordenadas de `longitude` y `latitude` de cada aeropuerto, así como el código `iata` del aeropuerto &mdash; por ejemplo, `'SEA'`para el [Aeropuerto Internacional de Seattle-Tacoma](https://es.wikipedia.org/wiki/Aeropuerto_Internacional_de_Seattle-Tacoma).

In [17]:
airports = data.airports.url
airports

'https://vega.github.io/vega-datasets/data/airports.csv'

Finalmente, usaremos datos de las rutas de vuelo, que contiene campos de origen (`origin`) y destino (`destination`) con el código IATA para el respectivo aeropuerto:

In [18]:
flights = data.flights_airport.url
flights

'https://vega.github.io/vega-datasets/data/flights-airport.csv'

Comencemos creando un mapa base usando la proyección `albersUsa` y luego añadimos una capa que grafica marcadores `circle` para cada aeropuerto:

In [19]:
alt.layer(
    alt.Chart(alt.topo_feature(usa, 'states')).mark_geoshape(
        fill='#ddd', stroke='#fff', strokeWidth=1
    ),
    alt.Chart(airports).mark_circle(size=9).encode(
        latitude='latitude:Q',
        longitude='longitude:Q',
        tooltip='iata:N'
    )
).project(
    type='albersUsa'
).properties(
    width=900,
    height=500
).configure_view(
    stroke=None
)

alt.LayerChart(...)

¡Son muchos aeropuertos! Obviamente, no todos son grandes centros de operaciones.

Similar al conjunto de datos de códigos postales, los datos de los aeropuertos incluyen puntos que están fuera de los Estados Unidos continentales. Así que de nuevo vemos puntos en la esquina superior izquierda. Puede que queramos filtrar estos puntos, pero para hacerlo primero necesitamos saber más sobre ellos.

_¡Actualizar la proyección del mapa de arriba a `albers` &ndash; deja de lado el comportamiento idiosincrásico de `albersUsa` &ndash; para que se revele la ubicación real de estos puntos adicionales!_

Ahora, en lugar de mostrar todos los aeropuertos de manera indiferenciada, identifiquemos los principales centros de operaciones considerando el número total de rutas que se originan en cada aeropuerto. Usaremos el conjunto de datos `routes` como nuestra principal fuente de datos: contiene una lista de rutas de vuelo que podemos agregar para contar el número de rutas de cada aeropuerto de origen (`origin`).

¡Sin embargo, los datos `routes` no incluyen las _ubicaciones (locations)_ de los aeropuertos! Para usar los datos `routes` con las ubicaciones, necesitamos una nueva transformación de datos: `lookup`. La transformación `lookup` toma un valor de campo en un conjunto de datos primario y lo usa como una _llave (key)_ para buscar información relacionada en otra tabla. En este caso, queremos comparar el código del aeropuerto de origen en nuestra base de datos de rutas con el campo `iata` de los datos `airports`, y luego extraer los campos correspondientes de `latitude` y `longitude`.

In [20]:
alt.layer(
    alt.Chart(alt.topo_feature(usa, 'states')).mark_geoshape(
        fill='#ddd', stroke='#fff', strokeWidth=1
    ),
    alt.Chart(flights).mark_circle().transform_aggregate(
        groupby=['origin'],
        routes='count()'
    ).transform_lookup(
        lookup='origin',
        from_=alt.LookupData(data=airports, key='iata',
                             fields=['state', 'latitude', 'longitude'])
    ).transform_filter(
        'datum.state !== "PR" && datum.state !== "VI"'
    ).encode(
        latitude='latitude:Q',
        longitude='longitude:Q',
        tooltip=['origin:N', 'routes:Q'],
        size=alt.Size('routes:Q', scale=alt.Scale(range=[0, 1000]), legend=None),
        order=alt.Order('routes:Q', sort='descending')
    )
).project(
    type='albersUsa'
).properties(
    width=900,
    height=500
).configure_view(
    stroke=None
)

alt.LayerChart(...)

_¿Qué aeropuertos de EE.UU. tienen el mayor número de rutas de salida?_

Ahora que podemos ver los aeropuertos, podrías desear interactuar con ellos para entender mejor la estructura de la red de tráfico aéreo. Podemos añadir una capa de marcadore `rule` para representar las rutas de los aeropuertos de `origen (origin)` a los aeropuertos de `destino (destination)`, lo que requiere dos transformaciones `lookup` para recuperar las coordenadas de cada punto final. Además, podemos usar una selección `single` para filtrar estas rutas, de tal manera que sólo se muestren las rutas que se originan en el aeropuerto seleccionado actualmente.

_A partir del mapa estático de arriba, ¿puedes construir una versión interactiva? Siéntete libre de saltarte el código de abajo para ver en el mapa interactivo primero y piensa en cómo podrías construirlo por tu cuenta!_

In [21]:
# selección interactiva para el aeropuerto de origen
# seleccion el aeropuerto más cercano al cursor del mouse
origin = alt.selection_single(
    on='mouseover', nearest=True,
    fields=['origin'], empty='none'
)

# la referencia de los datos compartidos para la transformación lookup
foreign = alt.LookupData(data=airports, key='iata',
                         fields=['latitude', 'longitude'])
    
alt.layer(
    # mapa base de los Estados Unidos
    alt.Chart(alt.topo_feature(usa, 'states')).mark_geoshape(
        fill='#ddd', stroke='#fff', strokeWidth=1
    ),
    # líneas de ruta desde el aeropuerto de origen seleccionado hasta los aeropuertos de destino
    alt.Chart(flights).mark_rule(
        color='#000', opacity=0.35
    ).transform_filter(
        origin # filtrar sólo al origen seleccionado
    ).transform_lookup(
        lookup='origin', from_=foreign # origen lat/lon
    ).transform_lookup(
        lookup='destination', from_=foreign, as_=['lat2', 'lon2'] # destino lat/lon
    ).encode(
        latitude='latitude:Q',
        longitude='longitude:Q',
        latitude2='lat2',
        longitude2='lon2',
    ),
    # El tamaño de los aeropuertos por el número de rutas de salida
    # 1. Conjunto de datos agregados de vuelos y aeropuertos
    # 2. Buscar datos de ubicación del conjunto de datos airport
    # 3. Eliminar Puerto Rico (PR) y las Islas Vírgenes (VI)
    alt.Chart(flights).mark_circle().transform_aggregate(
        groupby=['origin'],
        routes='count()'
    ).transform_lookup(
        lookup='origin',
        from_=alt.LookupData(data=airports, key='iata',
                             fields=['state', 'latitude', 'longitude'])
    ).transform_filter(
        'datum.state !== "PR" && datum.state !== "VI"'
    ).add_selection(
        origin
    ).encode(
        latitude='latitude:Q',
        longitude='longitude:Q',
        tooltip=['origin:N', 'routes:Q'],
        size=alt.Size('routes:Q', scale=alt.Scale(range=[0, 1000]), legend=None),
        order=alt.Order('routes:Q', sort='descending') # colocar arriba los círculos pequeños
    )
).project(
    type='albersUsa'
).properties(
    width=900,
    height=500
).configure_view(
    stroke=None
)

alt.LayerChart(...)

_¡Pasa el ratón sobre el mapa para probar la red de vuelos!_

## Choropleth Maps (Mapas coropléticos)

Un [mapa coroplético](https://es.wikipedia.org/wiki/Mapa_coroplético) utiliza regiones sombreadas o con textura para visualizar los valores de los datos. Los mapas de símbolos de tamaño variable a menudo son más precisos de leer, ya que la gente tiende a ser mejor en la estimación de las diferencias proporcionales entre el área de los círculos que entre las tonalidades de color. No obstante, los mapas de coropléticos son populares en la práctica y particularmente útiles cuando demasiados símbolos se vuelven perceptiblemente abrumadores.

Por ejemplo, mientras que los Estados Unidos sólo tienen 50 estados, tienen miles de condados dentro de esos estados. Construyamos un mapa coroplético de la tasa de desempleo por condado, allá por el año de recesión de 2008. En algunos casos, los archivos GeoJSON o TopoJSON de entrada pueden incluir datos estadísticos que podemos visualizar directamente. En este caso, sin embargo, tenemos dos archivos: nuestro archivo TopoJSON que incluye características de los límites de los condados (`usa`), y un archivo de texto separado que contiene estadísticas de desempleo:

In [22]:
unemp = data.unemployment.url
unemp

'https://vega.github.io/vega-datasets/data/unemployment.tsv'

Para integrar nuestras fuentes de datos, tendremos que usar de nuevo la transformación de `lookup`, aumentando nuestros datos de `geoshape` basados en TopoJSON con las tasas de desempleo. Podemos crear un mapa que incluya una codificación de color para el campo de tasa de desempleo, `rate`.

In [23]:
alt.Chart(alt.topo_feature(usa, 'counties')).mark_geoshape(
    stroke='#aaa', strokeWidth=0.25
).transform_lookup(
    lookup='id', from_=alt.LookupData(data=unemp, key='id', fields=['rate'])
).encode(
    alt.Color('rate:Q',
              scale=alt.Scale(domain=[0, 0.3], clamp=True), 
              legend=alt.Legend(format='%')),
    alt.Tooltip('rate:Q', format='.0%')
).project(
    type='albersUsa'
).properties(
    width=900,
    height=500
).configure_view(
    stroke=None
)

alt.Chart(...)

*Examina la tasa de desempleo por condado. Los valores mayores en Michigan podrían relacionarse con la industria automotriz. Los condados de los estados en ["Las Grandes Llanuras"](https://es.wikipedia.org/wiki/Grandes_Llanuras) y los montañosos exhiben ambas bajas **y** altas tasas. ¿Si esta variación es significante, o si posiblemente es un [artificio producto de un muestreo bajo](https://medium.com/@uwdata/surprise-maps-showing-the-unexpected-e92b67398865)? Para explorar más, intenta cambiar la escala de dominio superior (por ejempolo a `0.2`) para ajustar el color.*

Una preocupación central de los mapas coropléticos es la elección de los colores. Arriba, usamos el esquema predeterminado de Altair `'yellowgreenblue'` para los mapas de calor. Abajo comparamos otros esquemas, incluyendo un esquema secuencial de un solo tono (`tealblues`) que varía sólo en iluminación, un esquema secuencial de varios tonos (`viridis`) que aumenta tanto la iluminación como el tono, y un esquema divergente (`blueorange`) que usa un punto medio de color blanco:

In [24]:
# función para generar la especificación de un mapa para el esquema de color previsto
def map_(scheme):
    return alt.Chart().mark_geoshape().project(type='albersUsa').encode(
        alt.Color('rate:Q', scale=alt.Scale(scheme=scheme), legend=None)
    ).properties(width=305, height=200)

alt.hconcat(
    map_('tealblues'), map_('viridis'), map_('blueorange'),
    data=alt.topo_feature(usa, 'counties')
).transform_lookup(
    lookup='id', from_=alt.LookupData(data=unemp, key='id', fields=['rate'])
).configure_view(
    stroke=None
).resolve_scale(
    color='independent'
)

alt.HConcatChart(...)

_¿Cuál esquema de color consideras más efectivo? ¿Por qué lo crees? Modifica los mapas anteriores usando otros esquemas disponibles, como están descritos en la [documentación de Vega de los esquemas de colores](https://vega.github.io/vega/docs/schemes/)._

## Cartographic Projections (Proyecciones cartográficas)

Ahora que tenemos algo de experiencia creando mapas, acerquémonos a las proyecciones cartográficas. Como se explica en [Wikipedia](https://en.wikipedia.org/wiki/Map_projection),

> _All map projections necessarily distort the surface in some fashion. Depending on the purpose of the map, some distortions are acceptable and others are not; therefore, different map projections exist in order to preserve some properties of the sphere-like body at the expense of other properties._

**Traducción:**
>_Todas las proyeccciones de mapas distorsionan la superficie de alguna forma. Dependiendo del propósito del mapa, algunas distorsiones on aceptables y otras no; por lo tanto, existen diferentes proyeecciones de mapas con el objetivo de preservar algunas propiedades del cuerpo esférico a expensas de otras propiedades._

Algunas de las propiedades que podríamos considerar incluyen:
- _Área_: ¿La proyección distorsiona los tamaños de las regiones?
- _Rumbo (bearing)_: ¿Corresponde una línea recta a una dirección constante de viaje?
- _La distancia_: ¿Las líneas de igual longitud corresponden a distancias iguales en el globo?
- _Forma_: ¿La proyección conserva las relaciones espaciales (ángulos) entre los puntos?

La selección de una proyección apropiada depende, por lo tanto, del caso de uso del mapa. Por ejemplo, si estamos evaluando el uso de la tierra y la extensión de los asuntos de la tierra, podríamos elegir una proyección de conservación del área. Si queremos visualizar las ondas de choque que emanan de un terremoto, podríamos enfocar el mapa en el epicentro del terremoto y preservar las distancias hacia afuera de ese punto. O, si queremos ayudar a la navegación, la preservación del rumbo y la forma puede ser más importante.

También podemos caracterizar las proyecciones en términos de la _superficie de proyección_. Las proyecciones cilíndricas, por ejemplo, proyectan puntos de la superficie de la esfera sobre un cilindro circundante; el cilindro "desenrollado" proporciona entonces nuestro mapa. Como describimos más adelante, podemos proyectar alternativamente sobre la superficie de un cono (proyecciones cónicas) o directamente sobre un plano plano plano (proyecciones azimutales).

*Primero construyamos nuestra intuición interactuando con una variedad de proyecciones* **[Abre el cuaderno de Proyecciones Cartográficas Vega-Lite en línea](https://observablehq.com/@vega/vega-lite-cartographic-projections).** Utiliza los controles de esa página para seleccionar una proyección y explorar los parámetros de la misma, como `scale` (zoom) y el despalazamiento x/y (panning). Los controles de rotación ([yaw, pitch, roll/guiñada, inclinación, balanceo](https://es.wikipedia.org/wiki/Ejes_del_avi%C3%B3n)) determinan la orientación del globo en relación con la superficie sobre la que se proyecta.*

### A Tour of Specific Projection Types (Un recorrido por tipos de proyección específicos)

[**Proyecciones cilíndricas**](https://en.wikipedia.org/wiki/Map_projection#Cylindrical) mapean la esfera en un cilindro circundante, luego desenrollan el cilindro. Si el eje principal del cilindro está orientado de norte a sur, los meridianos se trazan en líneas rectas. Las proyecciones [seudo cilíndricas](https://en.wikipedia.org/wiki/Map_projection#Pseudocylindrical) representan un meridiano central como una línea recta, con otros meridianos "doblándose" alejándose del centro.

In [25]:
minimap = map.properties(width=225, height=225)
alt.hconcat(
    minimap.project(type='equirectangular').properties(title='equirectangular'),
    minimap.project(type='mercator').properties(title='mercator'),
    minimap.project(type='transverseMercator').properties(title='transverseMercator'),
    minimap.project(type='naturalEarth1').properties(title='naturalEarth1')
).properties(spacing=10).configure_view(stroke=None)

alt.HConcatChart(...)

- [Equirectangular](https://es.wikipedia.org/wiki/Proyecci%C3%B3n_cil%C3%ADndrica_equidistante) (`equirectangular`): Escala directamente los valores de las coordenadas `lat`, `lon`.
- [Mercator](https://es.wikipedia.org/wiki/Proyecci%C3%B3n_de_Mercator) (`mercator`): Proyecta en un cilindro, usando directamente `lon`, pero sometiendo `lat` a una transformación no lineal. Las líneas rectas preservan los rodamientos constantes de la brújula ([rhumb lines](https://es.wikipedia.org/wiki/Loxodr%C3%B3mica)), haciendo que esta proyección se adapte bien a la navegación. Sin embargo, las líneas áreas en el extremo norte o sur pueden estar muy distorsionadas.
- [Transverse Mercator](https://en.wikipedia.org/wiki/Transverse_Mercator_projection) (`transverseMercator`): Una proyección de mercator, pero con el cilindro delimitador girado a un eje transversal. Mientras que la proyección estándar de Mercator tiene la mayor precisión a lo largo del ecuador, la proyección transversal de Mercator es más precisa a lo largo del meridiano central.

- [Natural Earth](https://en.wikipedia.org/wiki/Natural_Earth_projection) (`naturalEarth1`): Una proyección seudo cilíndrica diseñada para mostrar toda la Tierra en una sola vista.
<br/><br/>

[**Proyecciones cónicas**](https://en.wikipedia.org/wiki/Map_projection#Conic) Trazan la esfera en un cono, y luego desenrollan el cono en el plano. Las proyecciones cónicas están configuradas por dos _paralelos estándar_, que determinan dónde el cono intersecta el globo.

In [26]:
minimap = map.properties(width=180, height=130)
alt.hconcat(
    minimap.project(type='conicEqualArea').properties(title='conicEqualArea'),
    minimap.project(type='conicEquidistant').properties(title='conicEquidistant'),
    minimap.project(type='conicConformal', scale=35, translate=[90,65]).properties(title='conicConformal'),
    minimap.project(type='albers').properties(title='albers'),
    minimap.project(type='albersUsa').properties(title='albersUsa')
).properties(spacing=10).configure_view(stroke=None)

alt.HConcatChart(...)

- [Conic Equal Area](https://es.wikipedia.org/wiki/Proyecci%C3%B3n_de_Albers) (`conicEqualArea`): La proyección cónica que conserva el área. La forma y la distancia no se conservan, pero son más o menos exactas dentro de los paralelos estándar.
- [Conic Equidistant](https://en.wikipedia.org/wiki/Equidistant_conic_projection) (`conicEquidistant`): Proyección cónica que preserva la distancia a lo largo de los meridianos y paralelos estándar.
- [Conic Conformal](https://en.wikipedia.org/wiki/Lambert_conformal_conic_projection) (`conicConformal`): Proyección cónica que conserva la forma (ángulos locales), pero no el área o la distancia.
- [Albers](https://es.wikipedia.org/wiki/Proyecci%C3%B3n_de_Albers) (`albers`): Una variante de la proyección cónica de área igual con paralelos estándar optimizada para crear mapas de los Estados Unidos.
- [Albers USA](https://es.wikipedia.org/wiki/Proyecci%C3%B3n_de_Albers) (`albersUsa`): Una proyección híbrida para los 50 estados de los Estados Unidos de América. Esta proyección une tres proyecciones de Albers con diferentes parámetros para los Estados Unidos continentales, Alaska y Hawai.
<br/><br/>

[**Proyecciones acimutales**](https://en.wikipedia.org/wiki/Map_projection#Azimuthal_%28projections_onto_a_plane%29) traza la esfera directamente en un plano.

In [27]:
minimap = map.properties(width=180, height=180)
alt.hconcat(
    minimap.project(type='azimuthalEqualArea').properties(title='azimuthalEqualArea'),
    minimap.project(type='azimuthalEquidistant').properties(title='azimuthalEquidistant'),
    minimap.project(type='orthographic').properties(title='orthographic'),
    minimap.project(type='stereographic').properties(title='stereographic'),
    minimap.project(type='gnomonic').properties(title='gnomonic')
).properties(spacing=10).configure_view(stroke=None)

alt.HConcatChart(...)

- [Azimuthal Equal Area](https://en.wikipedia.org/wiki/Lambert_azimuthal_equal-area_projection) (`azimuthalEqualArea`): Proyecta con precisión el área en todas las partes del globo, pero no conserva la forma (ángulos locales).
- [Azimuthal Equidistant](https://en.wikipedia.org/wiki/Azimuthal_equidistant_projection) (`azimuthalEquidistant`): Preserva la distancia proporcional del centro de proyección a todos los demás puntos del globo.
- [Orthographic](https://en.wikipedia.org/wiki/Orthographic_projection_in_cartography) (`orthographic`): Proyecta un hemisferio visible en un plano distante. Coincide aproximadamente con una vista de la Tierra desde el espacio exterior.
- [Stereographic](https://en.wikipedia.org/wiki/Stereographic_projection) (`stereographic`): Conserva la forma, pero no el área o la distancia.
- [Gnomonic](https://en.wikipedia.org/wiki/Gnomonic_projection) (`gnomonic`): Proyecta la superficie de la esfera directamente sobre un plano tangente.
- [Great circles](https://en.wikipedia.org/wiki/Great_circle) alrededor de la Tierra se proyectan en líneas rectas, mostrando el camino más corto entre los puntos.
<br/><br/>

## Coda: Wrangling Geographic Data (Manipulando datos geográficos)

Los ejemplos anteriores proceden sobre todo de la colección de conjuntos de datos vega, que incluye datos geométricos (TopoJSON) y tabulares (aeropuertos, tasas de desempleo). Un desafío común para comenzar con la visualización geográfica es la recolección de los datos necesarios para tu tarea. Abundan los proveedores de datos, incluyendo servicios como el [United States Geological Survey](https://www.usgs.gov/products/data-and-tools/data-and-tools-topics) y [U.S. Census Bureau](https://www.census.gov/geo/maps-data/data/tiger-cart-boundary.html).

En muchos casos puede que tengas datos existentes con un componente geográfico, pero que requieren medidas o geometría adicionales. Para ayudarte a empezar, aquí tienes un flujo de trabajo:

1. Visita [Natural Earth Data](http://www.naturalearthdata.com/downloads/) y navega para seleccionar datos para las regiones y resoluciones de interés, Descarga el respectivo archivo zip.
2. Ve a [MapShaper](https://mapshaper.org/) y arrastra el archivo que descargaste adentro de la página. Revisa los datos y luego expórtalos a un archivo TopoJSON o GeoJSON.
3. Carga los datos exportados para usarlos con Altair!

Por supuesto, muchas otras herramientas &ndash; tanto de código libre como propietario &ndash; existen para trabajar con datos geográficos. Para más acerca de manipulación de datos y creación de mapas, mira la serie de tutoriales de Mike Bostock en [Command-Line Cartography](https://medium.com/@mbostock/command-line-cartography-part-1-897aa8f8ca2c).

## Resumen
En este momento, sólo hemos sumergido los dedos de los pies en las aguas de la cartografía. _(¿No esperabas que en un solo notebook impartieramos siglos de aprendizaje, o sí?_ Por ejemplo, dejamos intactos temas como [_cartogramas_](https://es.wikipedia.org/wiki/Cartograma) y la expresión de [_topografía_](https://es.wikipedia.org/wiki/Topograf%C3%ADa) &mdash; como en el libro iluminador de Imhof [_presentación de relieve cartográfico_](https://books.google.com/books?id=cVy1Ms43fFYC). Sin embargo, ahora debes estar bien equipado para crear una rica gama de geovisualizaciones. Para más información, el libro de MacEachren [_How Maps Work: Representation, Visualization, and Design_](https://books.google.com/books?id=xhAvN3B0CkUC) proporciona una valiosa visión general de la elaboración de mapas desde la perspectiva de la visualización de datos. 